# Trimming Bad Data

We are now going to trim data from the sequences that are of bad quality using a program called "Trimmomatic"

An example use of this program loos like this

## TrimmomaticSE small.fastq.gz small_trimmed -threads 8 SLIDINGWINDOW:4:20 MINLEN:75

In [1]:
MIAMIID = !echo $USER
MIAMIID = str(MIAMIID)
MIAMIID = MIAMIID[2:len(MIAMIID)-2]
print(MIAMIID)

schroe51


# Important:

We now need to install trimmomatic using the following commands in the terminal:

First, we will download the zip file containing the .jar file.

In [5]:
%cd /home/{MIAMIID}/software
!wget http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.36.zip

/home/schroe51/software
--2022-11-17 11:34:11--  http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.36.zip
Resolving www.usadellab.org... 199.195.140.36
Connecting to www.usadellab.org|199.195.140.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131275 (128K) [application/zip]
Saving to: ‘Trimmomatic-0.36.zip’

Trimmomatic-0.36.zi 100%[===================>] 128.20K   366KB/s    in 0.4s    

2022-11-17 11:34:12 (366 KB/s) - ‘Trimmomatic-0.36.zip’ saved [131275/131275]



Now we will unzip and remove the original zip file:

In [6]:
!unzip Trimmomatic-0.36.zip
!rm Trimmomatic-0.36.zip
!ls

Archive:  Trimmomatic-0.36.zip
   creating: Trimmomatic-0.36/
  inflating: Trimmomatic-0.36/LICENSE  
  inflating: Trimmomatic-0.36/trimmomatic-0.36.jar  
   creating: Trimmomatic-0.36/adapters/
  inflating: Trimmomatic-0.36/adapters/NexteraPE-PE.fa  
  inflating: Trimmomatic-0.36/adapters/TruSeq2-PE.fa  
  inflating: Trimmomatic-0.36/adapters/TruSeq2-SE.fa  
  inflating: Trimmomatic-0.36/adapters/TruSeq3-PE-2.fa  
  inflating: Trimmomatic-0.36/adapters/TruSeq3-PE.fa  
  inflating: Trimmomatic-0.36/adapters/TruSeq3-SE.fa  
FastQC	sratoolkit.3.0.0-centos_linux64  Trimmomatic-0.36


## Now we will run Trimmomatic on SRR24062655 so we can obtain a higher quality version of the sequences in the file. Keep in mind this is paired end data.

In [2]:
%cd /home/{MIAMIID}/test/raw_data/fastq/
!ls

/home/schroe51/test/raw_data/fastq
getFastqData.sh       SRR24062654.fastq.gz  SRR24062657.fastq.gz
SRR24062652.fastq.gz  SRR24062655.fastq.gz
SRR24062653.fastq.gz  SRR24062656.fastq.gz


In [4]:
import os
numOfCPUs = len(os.sched_getaffinity(0))
print(numOfCPUs)

24


In [7]:
!java -jar /home/{MIAMIID}/software/Trimmomatic-0.36/trimmomatic-0.36.jar SE SRR24062655.fastq.gz SRR24062655_trimmed.fastq.gz -threads {numOfCPUs} SLIDINGWINDOW:4:20 MINLEN:75

TrimmomaticSE: Started with arguments:
 SRR24062655.fastq.gz SRR24062655_trimmed.fastq.gz -threads 24 SLIDINGWINDOW:4:20 MINLEN:75
Quality encoding detected as phred33
^C


In [5]:
!ls *_trimmed.fastq

SRR5017135_trimmed.fastq


In [8]:
%mkdir -p /home/{MIAMIID}/test/trimmed_reads
%mv *_trimmed.fastq /home/{MIAMIID}/test/trimmed_reads

mv: cannot stat ‘*_trimmed.fastq’: No such file or directory


In [9]:
%cd /home/{MIAMIID}/test/trimmed_reads
!ls

/home/schroe51/test/trimmed_reads


## We can now use fastqc on the trimmed sequence to see if the overall quality is improved.

Please make sure that "/home/{MIAMIID}/software/FastQC/" is on the PATH, if NOT run the following commands:

In [10]:
%env PATH

'/software/R/4.1.2/bin:/software/gcc/9.2.0/bin:/software/kallisto/0.48.0/bin:/software/python/anaconda3/condabin:/software/python/anaconda3/bin:/usr/local/bin:/usr/local/sbin:/opt/moab/bin:/usr/local/bin:/usr/local/sbin:/opt/moab/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/bin:/usr/bin:/usr/X11R6/bin:/opt/puppetlabs/bin'

In [11]:
curPath = %env PATH
pathWithFastQC = "/home/" + MIAMIID + "/software/FastQC/:" + curPath
print(pathWithFastQC)

/home/schroe51/software/FastQC/:/software/R/4.1.2/bin:/software/gcc/9.2.0/bin:/software/kallisto/0.48.0/bin:/software/python/anaconda3/condabin:/software/python/anaconda3/bin:/usr/local/bin:/usr/local/sbin:/opt/moab/bin:/usr/local/bin:/usr/local/sbin:/opt/moab/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/bin:/usr/bin:/usr/X11R6/bin:/opt/puppetlabs/bin


In [12]:
import os
os.environ['PATH'] = pathWithFastQC

In [11]:
%%sh
fastqc SRR5017135_trimmed.fastq

Analysis complete for SRR5017135_trimmed.fastq


Started analysis of SRR5017135_trimmed.fastq
Approx 55% complete for SRR5017135_trimmed.fastq


In [12]:
%mkdir -p /home/{MIAMIID}/test/trimmed_reads/fastqc_trimmed_results
%mv /home/{MIAMIID}/test/trimmed_reads/*.html /home/{MIAMIID}/test/trimmed_reads/fastqc_trimmed_results
%mv /home/{MIAMIID}/test/trimmed_reads/*.zip /home/{MIAMIID}/test/trimmed_reads/fastqc_trimmed_results

### Now you can compare the results between the untrimmed and trimmed data quality by viewing the html files in the "/home/{MIAMIID}/test/trimmed_reads/fastqc_trimmed_results" folder

# We can process all of the files with trimmomatic using the following script: 

### This will take a considerable amount of time (~1 hr+, depending on CPU resources)

In [2]:
%cd /home/{MIAMIID}/test/raw_data/fastq
!ls

/home/schroe51/test/raw_data/fastq
getFastqData.sh       SRR24062654.fastq.gz  SRR24062657.fastq.gz
SRR24062652.fastq.gz  SRR24062655.fastq.gz  trimFastq.sh
SRR24062653.fastq.gz  SRR24062656.fastq.gz


In [5]:
sh = """
for infile in /home/""" + MIAMIID + """/test/raw_data/fastq/*.gz
 do
 base=$(basename --suffix=.fastq $infile)
 java -jar /home/""" + MIAMIID + """/software/Trimmomatic-0.36/trimmomatic-0.36.jar SE \
  -threads """ + str(numOfCPUs) + """ \
  ${infile} ${base}_trimmed.fastq \
  SLIDINGWINDOW:4:20 MINLEN:75
 done
"""
with open('trimFastq.sh', 'w') as file:
  file.write(sh)

In [ ]:
slurm = """
#!/bin/bash
#Comment -  to be submitted by: sbatch rna-seq-notebook-2.sh
#SBATCH --time=4:00:00          # 4 hours max
#SBATCH --nodes=1               # 1 compute node
#SBATCH --ntasks-per-node=24    # 24 cpu cores
#SBATCH --partition=batch       # requests a compute node
#SBATCH --job-name=rna-seq-notebook-2
#SBATCH --output=rna-seq-notebook-2.out

#Comment - batch job setup complete

#Comment – load a program module, for example Python

source loadAnaconda.sh

#Comment – path to execution directory, for example

cd $HOME/test/raw_data/fastq

#Comment – program execution line

source trimFastq.sh
"""
with open('rna-seq-notebook-2.sh') as slurmScript:
    slurmScript.write(slurm)

In [6]:
!sbatch rna-seq-notebook-2.sh

TrimmomaticSE: Started with arguments:
 -threads 24 /home/schroe51/test/raw_data/fastq/SRR24062652.fastq.gz SRR24062652.fastq.gz_trimmed.fastq SLIDINGWINDOW:4:20 MINLEN:75
Quality encoding detected as phred33
Input Reads: 34669058 Surviving: 32878975 (94.84%) Dropped: 1790083 (5.16%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -threads 24 /home/schroe51/test/raw_data/fastq/SRR24062653.fastq.gz SRR24062653.fastq.gz_trimmed.fastq SLIDINGWINDOW:4:20 MINLEN:75
Quality encoding detected as phred33
Input Reads: 35289489 Surviving: 34057091 (96.51%) Dropped: 1232398 (3.49%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -threads 24 /home/schroe51/test/raw_data/fastq/SRR24062654.fastq.gz SRR24062654.fastq.gz_trimmed.fastq SLIDINGWINDOW:4:20 MINLEN:75
Quality encoding detected as phred33
Input Reads: 34928074 Surviving: 33207411 (95.07%) Dropped: 1720663 (4.93%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arg

Now we move all the trimmed files to another folder for organization.

In [7]:
!ls *_trimmed.fastq

SRR24062652.fastq.gz_trimmed.fastq  SRR24062655.fastq.gz_trimmed.fastq
SRR24062653.fastq.gz_trimmed.fastq  SRR24062656.fastq.gz_trimmed.fastq
SRR24062654.fastq.gz_trimmed.fastq  SRR24062657.fastq.gz_trimmed.fastq


In [8]:
%mkdir -p /home/{MIAMIID}/test/trimmed_reads
%mv *_trimmed.fastq /home/{MIAMIID}/test/trimmed_reads

Now we can run fastqc on these files to see the quality reports

We can also remove all of the normal fastq.gz files since we have higher quality versions of them now.

In [13]:
%%sh
for trimmed_file in ~/test/trimmed_reads/*_trimmed.fastq
 do
 fastqc $trimmed_file
 done

Analysis complete for SRR24062652.fastq.gz_trimmed.fastq
Analysis complete for SRR24062653.fastq.gz_trimmed.fastq
Analysis complete for SRR24062654.fastq.gz_trimmed.fastq
Analysis complete for SRR24062655.fastq.gz_trimmed.fastq
Analysis complete for SRR24062656.fastq.gz_trimmed.fastq
Analysis complete for SRR24062657.fastq.gz_trimmed.fastq


Started analysis of SRR24062652.fastq.gz_trimmed.fastq
Approx 5% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 10% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 15% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 20% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 25% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 30% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 35% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 40% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 45% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 50% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 55% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 60% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 65% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 70% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 75% complete for SRR24062652.fastq.gz_trimmed.fastq
Approx 80% complete for SRR24062652.fastq.gz_trimmed.fastq
Ap

In [14]:
%mkdir -p /home/{MIAMIID}/test/trimmed_reads/fastqc_trimmed_results
%mv /home/{MIAMIID}/test/trimmed_reads/*.html /home/{MIAMIID}/test/trimmed_reads/fastqc_trimmed_results
%mv /home/{MIAMIID}/test/trimmed_reads/*.zip /home/{MIAMIID}/test/trimmed_reads/fastqc_trimmed_results

# Now you have trimmed sequences of higher quality than the original sequences.